# 100 Pytorch Object Detection Exercises

The goal of this collection is to offer a quick reference for both old and new users who are designing and prototyping object detection algorithms with pytorch. 

In [14]:
import torch
import numpy as np

#### Generate coordinates of anchor boxes
```python
base_size = 16 # pixel size in original image, also named stride or downsampling factor. 
scale_ls = [8, 16, 32] # under the same aspect ratio, scale = w_a/w_b or h_a/h_b
ratio_ls = [0.5, 1, 2] # under the same scale, ratio = height/width
```

In [15]:
# code reference: https://github.com/NVIDIA/retinanet-examples/blob/c1ba8e7a9ffa036a387387426682e1a7b58ec707/retinanet/box.py#L5

scale_ls = [8,16,32]
ratio_ls = [0.5,1,2] 
base_size = 16

scales = torch.FloatTensor(scale_ls).repeat(len(ratio_ls), 1) # shape: (len(ratio_ls), len(scale_ls)), (3,3)
scales = scales.transpose(0, 1).contiguous().view(-1, 1) # shape: (len(ratio_ls)*len(scale_ls), 1), (9,1)
ratios = torch.FloatTensor(ratio_ls).repeat(len(scale_ls)) # shape: (len(ratio_ls)*len(scale_ls),), (9,)

base_wh = torch.FloatTensor([base_size]).repeat(len(ratios), 2) # base width and height, shape: (9,2)
anchor_w = torch.round(torch.sqrt(base_wh[:, 0] * base_wh[:, 1] / ratios)) # anchor width, shape: (9,)
anchor_h = torch.round(anchor_w * ratios) # anchor height, shape: (9,)
anchor_wh = torch.stack((anchor_w, torch.round(anchor_w * ratios)), dim=1) # anchor width and height, shape: (9,2)
# calculate the relative center coordinates, then the diff is the xmin,ymin and xmax,ymax
xy_min = 0.5 * (base_wh - anchor_wh * scales) # 0.5*(base_wh-1) - 0.5*(anchor_wh-1)
xy_max = 0.5 * (base_wh + anchor_wh * scales) - 1 # 0.5*(base_wh-1) + 0.5*(anchor_wh-1)
anchor_coord = torch.cat([xy_min, xy_max], dim=1)
print(anchor_coord)

tensor([[ -84.,  -40.,   99.,   55.],
        [ -56.,  -56.,   71.,   71.],
        [ -36.,  -80.,   51.,   95.],
        [-176.,  -88.,  191.,  103.],
        [-120., -120.,  135.,  135.],
        [ -80., -168.,   95.,  183.],
        [-360., -184.,  375.,  199.],
        [-248., -248.,  263.,  263.],
        [-168., -344.,  183.,  359.]])
